In [4]:
import pandas as pd
import numpy as np
import re, nltk
from sklearn.feature_extraction.text import CountVectorizer        
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from collections import Counter

train_data_df=pd.read_csv("trening.csv", delimiter=";",header=None)
train_data_df.columns=['Type', 'Text']
TypeText=['pozytywny','negatywny','neutralny']


In [5]:
test_data_df=pd.read_csv("test.csv", delimiter=";",header=None)
test_data_df.columns=['Text']

In [6]:
train_data_df.head(1)

,Type,Text
0,0,jest super dzien i lubie nasze slonce


In [14]:
np.mean([len(s.split(" ")) for s in train_data_df.Text])

stemmer = PorterStemmer()

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    text = re.sub("[^a-zA-Z]", " ", text)
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

POLISH_STOP_WORDS=['i','w','z','a','oraz','do','na','byc','sie','jest','mam','mamy','po']
vectorizer = CountVectorizer(
    analyzer = 'word',
    tokenizer = tokenize,
    lowercase = True,
    # !!! zamien ponizsze swoja lista polskich stop words
    stop_words=POLISH_STOP_WORDS,    
    max_features = 85
)

corpus_data_features = vectorizer.fit_transform(
    train_data_df.Text.tolist() + test_data_df.Text.tolist())
corpus_data_features_nd = corpus_data_features.toarray()
corpus_data_features_nd.shape
vocab = vectorizer.get_feature_names_out()
dist = np.sum(corpus_data_features_nd, axis=0)
X_train, X_test, y_train, y_test  = train_test_split(
        corpus_data_features_nd[0:len(train_data_df)], 
        train_data_df.Type,
        train_size=0.85, 
        random_state=1234)
log_model = LogisticRegression()
log_model = log_model.fit(X=X_train, y=y_train)
y_pred = log_model.predict(X_test)


/home/zajecia/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/zajecia/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['mami'] not in stop_words.
  warnings.warn(


In [15]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      0.80      0.89         5
           1       0.67      1.00      0.80         2
           2       1.00      1.00      1.00         2

    accuracy                           0.89         9
   macro avg       0.89      0.93      0.90         9
weighted avg       0.93      0.89      0.89         9



In [16]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[4, 1, 0],
       [0, 2, 0],
       [0, 0, 2]])